In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
from glob import glob

In [2]:
with open('mampu.json') as fopen:
    data = json.load(fopen)
len(data)

12127

In [3]:
directory = 'data-mampu'
!mkdir {directory}
!du -hs {directory}

mkdir: cannot create directory ‘data-mampu’: File exists
1.4G	data-mampu


In [10]:
files = {}
k = 0
for d in data:
    for url in d['file_urls']:
        file, ext = os.path.splitext(url)
        if not len(ext):
            file = f'{k}'
        else:
            file = f'{k}{ext}'
        files[url] = os.path.join(directory, file)
        k += 1

In [13]:
def crawl(url, file):
    if os.path.exists(file):
        return
    
    try:
        if not url.startswith('http'):
            url = 'https://www.data.gov.my' + url
        r = requests.get(url)
        with open(file, 'wb') as fopen:
            fopen.write(r.content)
    except:
        pass

In [14]:
files_t = tuple(files.items())
files_t[0]

('/data/ms_MY/dataset/peruntukan-dan-belanja-jalan-luar-bandar-jalb/resource/9ac42a0e-0c3c-44f8-9e27-c967af68f363',
 'data-mampu/0')

In [15]:
from concurrent.futures import ThreadPoolExecutor, as_completed

max_worker = 10

for i in tqdm(range(0, len(files_t), max_worker)):
    b = files_t[i: i + max_worker]
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(crawl, url[0], url[1]): url for url in b}

    for future in as_completed(futures):
        future.result()

100%|███████████████████████████████████████| 4973/4973 [58:29<00:00,  1.42it/s]


In [16]:
!du -hs data-mampu

5.1G	data-mampu
